<h2> Running Bayes with a Normal prior</h2>
Now I make another set of Bauesian tests with a normal prior. WHy don't I do this in the other notebook? Because the tests kept crashing.

In [1]:
# Importing Things #
import pandas as pd
import numpy as np
import math as m
from scipy import stats
from scipy.stats import gennorm
from scipy.stats import gamma
import seaborn as sns
import matplotlib.pyplot as plt
import pymc as pm
import pickle
import os

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
gGaussMean = 3
gGaussSpread = 2.5
gGaussBeta = 3
prior_mean = 24
numTests=40
lift=1.03
resultFile = r"C:\Users\efree\BayesTests_Normal_v0.pkl"

In [3]:
def generate_dataset(N=48000, R=0.60, M=gGaussMean, S=gGaussSpread, B=gGaussBeta,lift=1.00):
    first_col = np.random.choice(['T', 'C'], size=N)
    second_col = np.zeros(N)
    non_zero_count = int((1 - R) * N)
    non_zero_indices = np.random.choice(N, size=non_zero_count, replace=False)
    random_values = gennorm.rvs(beta=B, loc=M, scale=S, size=non_zero_count)
    second_col[non_zero_indices] = np.exp(random_values)
    df = pd.DataFrame({
        'TestControl': first_col,
        'Revenue': second_col
    })
    df['Revenue']=df.apply(lambda x: x['Revenue']*lift if x['TestControl']=='T' else x['Revenue'],axis=1)

    return df

In [4]:
def hurdle_normal_test(data: pd.DataFrame, samples: int = 3000, tune: int = 1000, random_seed: int = 42):
    # Split the data into test and control groups
    Control_data = data[data['TestControl'] == 'C']['Revenue'].values
    Test_data = data[data['TestControl'] == 'T']['Revenue'].values

    with pm.Model() as model:
        # Priors for group means and standard deviations
        mu_control = pm.Normal('mu_control', mu=prior_mean, sigma=10)
        sigma_control = pm.HalfNormal('sigma_control', sigma=10)

        mu_test = pm.Normal('mu_test', mu=prior_mean, sigma=10)
        sigma_test = pm.HalfNormal('sigma_test', sigma=10)

        # Likelihoods for observed data
        control_obs = pm.Normal('control_obs', mu=mu_control, sigma=sigma_control, observed=Control_data)
        test_obs = pm.Normal('test_obs', mu=mu_test, sigma=sigma_test, observed=Test_data)

        # Deterministic variable for the difference in means
        delta = pm.Deterministic('delta', mu_test - mu_control)
        revenue_T = pm.Deterministic('revenue_T',mu_test)
        revenue_C = pm.Deterministic('revenue_C', mu_control)

        # Sampling
        trace = pm.sample(samples, tune=tune, random_seed=random_seed, return_inferencedata=True, chains=2, target_accept=0.9)

    # Calculate the probability that test mean > control mean
    delta_samples = trace.posterior['delta'].values.flatten()
    percent_chance_T_better = (delta_samples > 0).mean()

    return percent_chance_T_better

In [5]:
%%capture

for i in range(numTests):
    data1 = generate_dataset(lift=lift)
    t_mean = data1[data1['TestControl']=='T']['Revenue'].mean()
    c_mean = data1[data1['TestControl']=='C']['Revenue'].mean()
    t_statistic, p_value = stats.ttest_ind(data1[data1['TestControl']=='T']['Revenue'].values, data1[data1['TestControl']=='C']['Revenue'].values)
    try:
        tBayes_normal= hurdle_normal_test(data1)
    except:
        tBayes_normal=0.50
    if os.path.exists(resultFile):
        with open(resultFile, 'rb') as fileName:
          resultList = pickle.load(fileName)
        resultList.append([tBayes_normal, t_mean, c_mean, p_value])
        with open(resultFile, 'wb') as fileName:
            pickle.dump(resultList,fileName)
    else:
        resultList=[[tBayes_normal, t_mean, c_mean, p_value]]
        with open(resultFile, 'wb') as fileName:
            pickle.dump(resultList,fileName)       

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_control, sigma_control, mu_test, sigma_test]
Sampling 2 chains for 1_000 tune and 3_000 draw iterations (2_000 + 6_000 draws total) took 179 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_control, sigma_control, mu_test, sigma_test]
Sampling 2 chains for 1_000 tune and 3_000 draw iterations (2_000 + 6_000 draws total) took 173 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_control, sigma_control, mu_test, sigma_test]
Sampling 2 chains for 1_000 tune and 3_000 draw iterations (2_000 + 6_000 draws total) took 181 seconds.
We recommend running at least 4 chains for robust computation of convergence d

In [6]:
print("All Done!")

All Done!
